In [1]:
from astropy.io import fits
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
from scipy.interpolate import interp1d
from NonnegMFPy import nmf
from astropy.modeling import models
import astropy.units as u
import functools
from scipy.optimize import minimize
from astropy.stats import sigma_clip
from scipy import ndimage, misc

In [2]:
# plotting setting
plt.rcParams['xtick.direction'] = 'in'
plt.rcParams['ytick.direction'] = 'in'
plt.rcParams['mathtext.fontset']='stix'
plt.rcParams['lines.linewidth'] = 1
plt.rcParams['xtick.labelsize'] = 16
plt.rcParams['ytick.labelsize'] = 16
plt.rcParams['axes.labelsize'] = 20
plt.rcParams['axes.titlesize'] = 20
plt.rcParams['xtick.direction'] = 'in'
plt.rcParams['ytick.direction'] = 'in'
plt.rcParams['xtick.minor.visible'] = True
plt.rcParams['ytick.minor.visible'] = True
plt.rcParams['xtick.major.size'] = 8
plt.rcParams['ytick.major.size'] = 8
plt.rcParams['xtick.minor.size'] = 4
plt.rcParams['ytick.minor.size'] = 4
plt.rcParams['xtick.major.width'] = 2
plt.rcParams['ytick.major.width'] = 2
plt.rcParams['xtick.minor.width'] = 2
plt.rcParams['ytick.minor.width'] = 2
plt.rcParams['xtick.top'] = True
plt.rcParams['ytick.right'] = True
plt.rcParams['axes.spines.bottom']=True
plt.rcParams['axes.spines.top']=True
plt.rcParams['axes.spines.left']=True
plt.rcParams['axes.spines.right']=True
plt.rcParams['xtick.bottom']=True
plt.rcParams['xtick.labelbottom']=True
plt.rcParams['ytick.left']=True
plt.rcParams['ytick.labelleft']=True

# SDSS sample selection
## wavelength coverage: $3600 \sim 10400 {\rm \overset{\circ}{A}}$

## Lya $1216 {\rm \overset{\circ}{A}}$, $z_{\rm min} \approx \frac{3600}{1216} - 1=1.96$
## MgII $2796, 2803 {\rm \overset{\circ}{A}}$, $z_{\rm max} \approx \frac{10400}{2803} - 1 = 2.7$

### The redshift range is 1.96 - 2.7

## ref: Davies+18 $(2.09<z_{\rm pipe}<2.51)$
* https://ui.adsabs.harvard.edu/abs/2018ApJ...864..143D/abstract
* BAL_FLAG_VI = 0 (reject BAL QSOs) 
* ZWARNING=0(avoid objects with high uncertain redshifts)

### In DR 16 catalog: BAL_FLAG_VI = 0 <=> Lyke+2020 BAL_PROB = 0 


In [3]:
DR16Q = fits.open('DR16Q_v4.fits')

In [4]:
z_pipe = DR16Q[1].data['Z_PIPE']
BAL_PROB = DR16Q[1].data['BAL_PROB']
zwarning = DR16Q[1].data['ZWARNING']
plate = DR16Q[1].data['PLATE']
mjd = DR16Q[1].data['MJD']
fiberid = DR16Q[1].data['FIBERID']

In [5]:
critreia = ((z_pipe>2.09) & (z_pipe<2.51) & (BAL_PROB==0) & (zwarning==0))
useful_indices = np.where(critreia)

In [6]:
f = open('QSO_list.txt', 'w')
for i in useful_indices[0]:
    test_fiber_id = '%s' % fiberid[i]
    if len(test_fiber_id) == 1:
        test_fiber_id = '000' + test_fiber_id
    elif len(test_fiber_id) == 2:
        test_fiber_id = '00' + test_fiber_id
    elif len(test_fiber_id) == 3:
        test_fiber_id = '0' + test_fiber_id
    else:
        test_fiber_id = test_fiber_id
    name = '%s/spec-%s-%s-%s.fits\t %f\n' % (plate[i],plate[i], mjd[i], test_fiber_id, z_pipe[i])
    f.write(name)
f.close()